In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/resample1/original_test.csv
/kaggle/input/resample1/resampled_borderline_train.csv
/kaggle/input/resample1/resampled_nn_train.csv
/kaggle/input/resample1/resampled_adasyn_train.csv
/kaggle/input/resample1/resampled_tomek_train.csv
/kaggle/input/mmai823-ai-finance/cleaned_data.csv


In [2]:
import pandas as pd
cleaned_data = pd.read_csv("/kaggle/input/mmai823-ai-finance/cleaned_data.csv")
resampled_adasyn_train = pd.read_csv("/kaggle/input/resample1/resampled_adasyn_train.csv")
resampled_borderline_train = pd.read_csv("/kaggle/input/resample1/resampled_borderline_train.csv")
resampled_nn_train = pd.read_csv("/kaggle/input/resample1/resampled_nn_train.csv")
resampled_tomek_train = pd.read_csv("/kaggle/input/resample1/resampled_tomek_train.csv")

In [3]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras import optimizers
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report,  balanced_accuracy_score, roc_auc_score, accuracy_score
from keras.layers import Input, Dense, Dropout, Embedding, Flatten
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
from collections import defaultdict
from yellowbrick.classifier import ROCAUC

Using TensorFlow backend.


In [4]:
#df_target = cleaned_data[cleaned_data['BK']==1]
#df_non_bk=cleaned_data[cleaned_data['BK']==0]

In [5]:
#df_non_bk_dsample=df_non_bk.sample(n=1000, random_state=42)

In [6]:
#df=pd.concat([df_target,df_non_bk_dsample],ignore_index=True)

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split


cleaned_data_X=cleaned_data.drop(columns=['BK','Unnamed: 0'])
cleaned_data_y=cleaned_data['BK']


In [8]:
#original_test_x=original_test.drop(columns=['BK'])
#original_test_y=original_test['BK']

In [9]:
from sklearn.model_selection import train_test_split

seed=42

X_train, X_test, y_train, y_test = train_test_split(cleaned_data_X, cleaned_data_y, test_size=0.2, random_state=seed)

In [10]:
resampler = SMOTEENN(random_state=42)
X_res, y_res = resampler.fit_resample(X_train, y_train)

resampler = SMOTETomek(random_state=42)
X_tom, y_tom = resampler.fit_resample(X_train, y_train)

resampler = BorderlineSMOTE(random_state=42)
X_blsmote, y_blsmote = resampler.fit_resample(X_train, y_train)

resampler = ADASYN(random_state=42)
X_adasyn, y_adasyn = resampler.fit_resample(X_train, y_train)


In [11]:
from sklearn.metrics import roc_auc_score

In [ ]:
# Base model
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='adam', alpha=1, hidden_layer_sizes=(13, 13, 13 ,13), activation='relu',verbose=True, random_state=42)

clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

# Balanced Accuracy Score
rf_score = balanced_accuracy_score(y_test, predictions)
# AUC Score
rf_auc = roc_auc_score(y_test, predictions)

print('Benchmark Balanced Accuracy: ' + str(rf_score))
print('Benchmark AUC Score:' + str(rf_auc))
print(classification_report(y_test, predictions, digits=3))

In [27]:
# fit model no training data
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='adam', alpha=1, hidden_layer_sizes=(13, 13, 13 ,13), activation='relu',verbose=True, random_state=42 )

clf.fit(X_res, y_res)

Iteration 1, loss = 0.53799124
Iteration 2, loss = 0.44191644
Iteration 3, loss = 0.41636037
Iteration 4, loss = 0.39920958
Iteration 5, loss = 0.38691311
Iteration 6, loss = 0.37645191
Iteration 7, loss = 0.37051237
Iteration 8, loss = 0.36558766
Iteration 9, loss = 0.36198667
Iteration 10, loss = 0.35803030
Iteration 11, loss = 0.35437309
Iteration 12, loss = 0.35165490
Iteration 13, loss = 0.34929417
Iteration 14, loss = 0.34636620
Iteration 15, loss = 0.34538074
Iteration 16, loss = 0.34313000
Iteration 17, loss = 0.34188208
Iteration 18, loss = 0.34015666
Iteration 19, loss = 0.33936522
Iteration 20, loss = 0.33838274
Iteration 21, loss = 0.33726716
Iteration 22, loss = 0.33629030
Iteration 23, loss = 0.33541018
Iteration 24, loss = 0.33414064
Iteration 25, loss = 0.33374998
Iteration 26, loss = 0.33283385
Iteration 27, loss = 0.33231614
Iteration 28, loss = 0.33157570
Iteration 29, loss = 0.33029310
Iteration 30, loss = 0.32990188
Iteration 31, loss = 0.32880742
Iteration 32, los

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.5, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(13, 13, 13, 13), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=42, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=True, warm_start=False)

In [28]:
predictions = clf.predict(X_test)

# Balanced Accuracy Score
rf_score = balanced_accuracy_score(y_test, predictions)
# AUC Score
rf_auc = roc_auc_score(y_test, predictions)

print('Benchmark Balanced Accuracy: ' + str(rf_score))
print('Benchmark AUC Score:' + str(rf_auc))
print(classification_report(y_test, predictions, digits=3))

Benchmark Balanced Accuracy: 0.7701455552954031
Benchmark AUC Score:0.7701455552954031
              precision    recall  f1-score   support

           0      0.998     0.874     0.931     17084
           1      0.033     0.667     0.063       111

    accuracy                          0.872     17195
   macro avg      0.515     0.770     0.497     17195
weighted avg      0.991     0.872     0.926     17195



In [29]:
from sklearn.metrics import confusion_matrix

print (confusion_matrix(y_test, predictions))


[[14925  2159]
 [   37    74]]


In [ ]:
# fit model no training data
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='adam', alpha=1, hidden_layer_sizes=(13, 13, 13 ,13), activation='relu',verbose=True, random_state=42)

clf.fit(X_tom, y_tom)

predictions = clf.predict(X_test)

# Balanced Accuracy Score
rf_score = balanced_accuracy_score(y_test, predictions)
# AUC Score
rf_auc = roc_auc_score(y_test, predictions)

print('Benchmark Balanced Accuracy: ' + str(rf_score))
print('Benchmark AUC Score:' + str(rf_auc))
print(classification_report(y_test, predictions, digits=3))

In [ ]:
# fit model no training data
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='adam', alpha=1, hidden_layer_sizes=(13, 13, 13 ,13), activation='relu',verbose=True, random_state=42)

clf.fit(X_blsmote, y_blsmote)

predictions = clf.predict(X_test)

# Balanced Accuracy Score
rf_score = balanced_accuracy_score(y_test, predictions)
# AUC Score
rf_auc = roc_auc_score(y_test, predictions)

print('Benchmark Balanced Accuracy: ' + str(rf_score))
print('Benchmark AUC Score:' + str(rf_auc))
print(classification_report(y_test, predictions, digits=3))

predictions = clf.predict(X_test)


In [ ]:
# fit model no training data
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='adam', alpha=1, hidden_layer_sizes=(13, 13, 13 ,13), activation='relu',verbose=True, random_state=42)

clf.fit(X_adasyn, y_adasyn)

predictions = clf.predict(X_test)

# Balanced Accuracy Score
rf_score = balanced_accuracy_score(y_test, predictions)
# AUC Score
rf_auc = roc_auc_score(y_test, predictions)

print('Benchmark Balanced Accuracy: ' + str(rf_score))
print('Benchmark AUC Score:' + str(rf_auc))
print(classification_report(y_test, predictions, digits=3))